<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [99]:
##### Your Code Here #####
import pandas as pd 

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [100]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,6469-MRVET,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [101]:
import numpy as np
def wrangle(X):
    X = X.copy()
    
    # Need to convert empty space to NAN value and then convert TotalCharges column from string to float
    X['TotalCharges'] = X['TotalCharges'].replace(" ", np.NAN).astype('float')
    
    return X

In [102]:
from sklearn.model_selection import train_test_split
target = 'Churn'
features = df.columns.drop([target, 'customerID'])

x_train, x_test, y_train, y_test = train_test_split(df[features], df[target]) 

In [103]:
x_train = wrangle(x_train)
x_test = wrangle(x_test)

In [104]:
y_train = y_train.map({'No': 0, 'Yes': 1})
y_test = y_test.map({'No': 0, 'Yes': 1})

In [105]:
x_train_trans.shape

(5282, 45)

In [106]:
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

transformers = make_pipeline(
    ce.OneHotEncoder(),
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

x_train_trans = transformers.fit_transform(x_train)
x_test_trans =transformers.transform(x_test)

#### Baseline

In [107]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Important Hyperparameters
inputs = x_train_trans.shape[1]
epochs = 75
batch_size = 40


# Create Model
model = Sequential()
model.add(Dense(46, activation='relu', input_shape=(inputs,)))
model.add(Dense(46, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit Model
model.fit(x_train_trans, y_train, 
          validation_data=(x_test_trans,y_test), 
          epochs=epochs,
          batch_size=batch_size 
         )

Train on 5282 samples, validate on 1761 samples
Epoch 1/75
5282/5282 [==============================] - 10s 2ms/sample - loss: 0.4675 - acc: 0.7651 - val_loss: 0.4401 - val_acc: 0.7950
Epoch 2/75
5282/5282 [==============================] - 2s 325us/sample - loss: 0.4200 - acc: 0.7991 - val_loss: 0.4327 - val_acc: 0.7927
Epoch 3/75
5282/5282 [==============================] - 2s 315us/sample - loss: 0.4105 - acc: 0.8061 - val_loss: 0.4358 - val_acc: 0.8012
Epoch 4/75
5282/5282 [==============================] - 2s 326us/sample - loss: 0.4072 - acc: 0.8076 - val_loss: 0.4307 - val_acc: 0.8030
Epoch 5/75
5282/5282 [==============================] - 2s 315us/sample - loss: 0.4010 - acc: 0.8084 - val_loss: 0.4296 - val_acc: 0.8018
Epoch 6/75
5282/5282 [==============================] - 2s 339us/sample - loss: 0.3994 - acc: 0.8126 - val_loss: 0.4326 - val_acc: 0.7973
Epoch 7/75
5282/5282 [==============================] - 2s 329us/sample - loss: 0.3938 - acc: 0.8129 - val_loss: 0.4334 - val

#### Hyperparameter Tuning

In [90]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(46, activation='relu', input_shape=(inputs,)))
    model.add(Dense(46, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [100, 150, 175, 185, 200],
              'epochs': [20]}
# Create Grid Search
grid = GridSearchCV(cv=5, estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train_trans, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.8042408163147594 using {'batch_size': 185, 'epochs': 20}
Means: 0.7955319851535326, Stdev: 0.013699782196217305 with: {'batch_size': 100, 'epochs': 20}
Means: 0.7983718377561653, Stdev: 0.00906911058550322 with: {'batch_size': 150, 'epochs': 20}
Means: 0.7981824927012247, Stdev: 0.012446917913361475 with: {'batch_size': 175, 'epochs': 20}
Means: 0.8042408163147594, Stdev: 0.010213712746552974 with: {'batch_size': 185, 'epochs': 20}
Means: 0.7968572458052716, Stdev: 0.008608178576916306 with: {'batch_size': 200, 'epochs': 20}


In [98]:
def create_model(optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(46, activation='relu', input_shape=(inputs,)))
    model.add(Dense(46, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
param_grid = {'batch_size': [185],
              'epochs': [20],
             'optimizer': ['adam', 'sgd', 'nadam', 'adagrad']}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train_trans, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

W0123 16:17:40.066826  3392 deprecation.py:506] From C:\Users\Elina\Anaconda3\envs\U4-S2-NN\lib\site-packages\tensorflow\python\keras\optimizer_v2\adagrad.py:105: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Best: 0.7972358964194587 using {'batch_size': 185, 'epochs': 5, 'optimizer': 'nadam'}
Means: 0.7896629860571776, Stdev: 0.013144809349408353 with: {'batch_size': 185, 'epochs': 5, 'optimizer': 'adam'}
Means: 0.7413858429517136, Stdev: 0.011698256681521437 with: {'batch_size': 185, 'epochs': 5, 'optimizer': 'sgd'}
Means: 0.7972358964194587, Stdev: 0.015279685953678948 with: {'batch_size': 185, 'epochs': 5, 'optimizer': 'nadam'}
Means: 0.7624006022652277, Stdev: 0.005494197676248694 with: {'batch_size': 185, 'epochs': 5, 'optimizer': 'adagrad'}


In [94]:
# define the grid search parameters
param_grid = {'batch_size': [185],
              'epochs': [5, 8, 10, 13]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x_train_trans, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7989397918081157 using {'batch_size': 185, 'epochs': 5}
Means: 0.7989397918081157, Stdev: 0.019667974396192257 with: {'batch_size': 185, 'epochs': 5}
Means: 0.7979931988778416, Stdev: 0.013983014557568823 with: {'batch_size': 185, 'epochs': 8}
Means: 0.797235899037459, Stdev: 0.012439649532485308 with: {'batch_size': 185, 'epochs': 10}
Means: 0.7947746813297272, Stdev: 0.011990713865128785 with: {'batch_size': 185, 'epochs': 13}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?